In [5]:
!pip install torch==2.3.0 torchtext==0.18.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.1/779.1 MB 2.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 64.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.1 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 80.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 69.6 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 40.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.0 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 13.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 30.8 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 3.4 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 8.3 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━

я пользуюсь кодом из:
https://pytorch.org/tutorials/beginner/torchtext_translation_tutorial.html
https://github.com/pytorch/examples/tree/main/language_translation

In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import torchtext
from torchtext.vocab import build_vocab_from_iterator
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator
#import sacrebleu
import wandb
from tqdm import tqdm
import math
from time import time

/usr/local/lib/python3.10/dist-packages/torchtext/vocab/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/utils.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated and the last released version will be 0.18 (this one). You can silence this warning by calling the following at the beginnign of your scripts: `import torchtext; torchtext.disable_torchtext_deprecation_warning()`
  warnings.warn(torchtext._TORCHTEXT_DEPRECATION_MSG)
/usr/local/lib/python3.10/dist-packages/torchtext/data/__init__.py:4: UserWarning: 
/!\ IMPORTANT WARNING ABOUT TORCHTEXT STATUS /!\ 
Torchtext is deprecated

In [3]:
wandb.login(key="6ec916c1b5b8de7f9c11e7aff05f7f98c8166837")

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: adromashkina (adromashkina-hse). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [4]:
wandb.init(project="translation-de-en", name="run-small-long")

wandb: Tracking run with wandb version 0.19.1
wandb: Run data is saved locally in /kaggle/working/wandb/run-20250309_193758-5xpuhgh2
wandb: Run `wandb offline` to turn off syncing.
wandb: Syncing run run-small-long
wandb: ⭐️ View project at https://wandb.ai/adromashkina-hse/translation-de-en
wandb: 🚀 View run at https://wandb.ai/adromashkina-hse/translation-de-en/runs/5xpuhgh2


In [7]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [8]:
def load_data(path):
    with open(path, encoding='utf-8') as f:
        return [line.strip().split() for line in f]

In [7]:
# def write_file_part(file_old, file_new, length):
#   old_file = ''.join(open(file_old).readlines()[:length])
#   open(file_new, 'w').write(old_file)

# data_folder = '/kaggle/input/en-german/data/'
# texts_folder = '/kaggle/working/'
# new_train_len = int(len(open(f'{data_folder}train.de-en.en').readlines()) * 0.1)
# write_file_part(f'{data_folder}train.de-en.en', f'{texts_folder}train_0.1.de-en.en', new_train_len)
# write_file_part(f'{data_folder}train.de-en.de', f'{texts_folder}train_0.1.de-en.de', new_train_len)

# new_val_len = int(len(open(f'{data_folder}val.de-en.en').readlines()) * 0.1)
# write_file_part(f'{data_folder}val.de-en.en', f'{texts_folder}val_0.1.de-en.en', new_val_len)
# write_file_part(f'{data_folder}val.de-en.de', f'{texts_folder}val_0.1.de-en.de', new_val_len)

In [8]:
# train_src = load_data("/kaggle/working/train_0.1.de-en.de")
# train_tgt = load_data("/kaggle/working/train_0.1.de-en.en")
# val_src = load_data("/kaggle/working/val_0.1.de-en.de")
# val_tgt = load_data("/kaggle/working/val_0.1.de-en.en")
# test_data = load_data("/kaggle/input/en-german/data/test1.de-en.de")

In [9]:
train_src = load_data("/kaggle/input/en-german/data/train.de-en.de")
train_tgt = load_data("/kaggle/input/en-german/data/train.de-en.en")
val_src = load_data("/kaggle/input/en-german/data/val.de-en.de")
val_tgt = load_data("/kaggle/input/en-german/data/val.de-en.en")
test_data = load_data("/kaggle/input/en-german/data/test1.de-en.de")

In [10]:
# import random
# all_src = train_src + val_src
# all_tgt = train_tgt + val_tgt

# data_pairs = list(zip(all_src, all_tgt))
# random.shuffle(data_pairs)

# train_size = int(0.3 * len(data_pairs))  
# #val_size = len(data_pairs) - train_size  

# # Разделяем на train и val
# train_data = data_pairs[:train_size]
# val_data = data_pairs[train_size:int(1.5*train_size)]

# # Разделяем обратно на src и tgt
# train_src, train_tgt = zip(*train_data)
# val_src, val_tgt = zip(*val_data)

# # Преобразуем обратно в списки
# train_src = list(train_src)
# train_tgt = list(train_tgt)
# val_src = list(val_src)
# val_tgt = list(val_tgt)

# # Проверяем размеры данных
# print(f"Train data size: {len(train_src)} -> {len(train_tgt)}")
# print(f"Validation data size: {len(val_src)} -> {len(val_tgt)}")

==================================================================================================

In [10]:
def _yield_tokens(data):
    for sentence in data:
        yield sentence


def get_data(train_src, train_tgt, val_src, val_tgt, test_data):
    src_lang = "de"
    tgt_lang = "en"
    
    train_src = load_data("/kaggle/input/en-german/data/train.de-en.de")
    train_tgt = load_data("/kaggle/input/en-german/data/train.de-en.en")
    val_src = load_data("/kaggle/input/en-german/data/val.de-en.de")
    val_tgt = load_data("/kaggle/input/en-german/data/val.de-en.en")
    test_data = load_data("/kaggle/input/en-german/data/test1.de-en.de")


    special_symbols = {
        "<unk>":0,
        "<pad>":1,
        "<bos>":2,
        "<eos>":3
    }

    
    src_vocab = build_vocab_from_iterator(
        _yield_tokens(train_src),
        min_freq=5,
        specials=list(special_symbols.keys()),
        special_first=True
    )

    tgt_vocab = build_vocab_from_iterator(
        _yield_tokens(train_tgt),
        min_freq=5,
        specials=list(special_symbols.keys()),
        special_first=True
    )

    src_vocab.set_default_index(special_symbols["<unk>"])
    tgt_vocab.set_default_index(special_symbols["<unk>"])

    def _seq_transform(*transforms):
        def func(txt_input):
            for transform in transforms:
                txt_input = transform(txt_input)
            return txt_input
        return func


    # Function to add BOS/EOS and create tensor for input sequence indices
    def _tensor_transform(token_ids):
        return torch.cat(
            (torch.tensor([special_symbols["<bos>"]]),
             torch.tensor(token_ids),
             torch.tensor([special_symbols["<eos>"]]))
        )

    src_lang_transform = _seq_transform(src_vocab, _tensor_transform)
    tgt_lang_transform = _seq_transform(tgt_vocab, _tensor_transform)


    def _collate_fn(batch):
        src_batch, tgt_batch = [], []
        for src_sample, tgt_sample in batch:
            #print(src_sample)
            #print('---')
            #print(tgt_sample)
            src_batch.append(src_lang_transform(src_sample))
            tgt_batch.append(tgt_lang_transform(tgt_sample))

        src_batch = pad_sequence(src_batch, padding_value=special_symbols["<pad>"])
        tgt_batch = pad_sequence(tgt_batch, padding_value=special_symbols["<pad>"])
        return src_batch, tgt_batch
    
    BATCH_SIZE = 128
    train_iterator = list(zip(train_src, train_tgt))
    valid_iterator = list(zip(val_src, val_tgt))
    train_dataloader = DataLoader(train_iterator, batch_size=BATCH_SIZE, collate_fn=_collate_fn)
    valid_dataloader = DataLoader(valid_iterator, batch_size=BATCH_SIZE, collate_fn=_collate_fn)

    return train_dataloader, valid_dataloader, src_vocab, tgt_vocab, src_lang_transform, tgt_lang_transform, special_symbols

In [11]:
class PositionalEncoding(nn.Module):
    def __init__(
        self,
        emb_size,
        dropout,
        maxlen=128
    ):
        super(PositionalEncoding, self).__init__()
        den = torch.exp(- torch.arange(0, emb_size, 2)* math.log(10000) / emb_size)
        pos = torch.arange(0, maxlen).reshape(maxlen, 1)
        pos_embedding = torch.zeros((maxlen, emb_size))
        pos_embedding[:, 0::2] = torch.sin(pos * den)
        pos_embedding[:, 1::2] = torch.cos(pos * den)
        pos_embedding = pos_embedding.unsqueeze(-2)

        self.dropout = nn.Dropout(dropout)
        self.register_buffer('pos_embedding', pos_embedding)

    def forward(self, token_embedding):
        return self.dropout(token_embedding + self.pos_embedding[:token_embedding.size(0), :])

In [12]:
class Translator(nn.Module):
    def __init__(
            self,
            num_encoder_layers,
            num_decoder_layers,
            embed_size,
            num_heads,
            src_vocab_size,
            tgt_vocab_size,
            dim_feedforward,
            dropout
        ):
        super(Translator, self).__init__()

        self.src_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.tgt_embedding = nn.Embedding(tgt_vocab_size, embed_size)

        self.pos_enc = PositionalEncoding(embed_size, dropout)

        self.transformer = nn.Transformer(
            d_model=embed_size,
            nhead=num_heads,
            num_encoder_layers=num_encoder_layers,
            num_decoder_layers=num_decoder_layers,
            dim_feedforward=dim_feedforward,
            dropout=dropout
        )

        self.ff = nn.Linear(embed_size, tgt_vocab_size)

        self._init_weights()

    def _init_weights(self):
        for p in self.parameters():
            if p.dim() > 1:
                nn.init.xavier_uniform_(p)

    def forward(self, src, trg, src_mask, tgt_mask, src_padding_mask, tgt_padding_mask, memory_key_padding_mask):

        src_emb = self.pos_enc(self.src_embedding(src))
        tgt_emb = self.pos_enc(self.tgt_embedding(trg))

        outs = self.transformer(
            src_emb,
            tgt_emb,
            src_mask,
            tgt_mask,
            None,
            src_padding_mask,
            tgt_padding_mask,
            memory_key_padding_mask
        )

        return self.ff(outs)

    def encode(self, src, src_mask):

        embed = self.src_embedding(src)

        pos_enc = self.pos_enc(embed)

        return self.transformer.encoder(pos_enc, src_mask)

    def decode(self, tgt, memory, tgt_mask):
        
        embed = self.tgt_embedding(tgt)

        pos_enc = self.pos_enc(embed)

        return self.transformer.decoder(pos_enc, memory, tgt_mask)

In [13]:
def generate_square_subsequent_mask(size, device):
    mask = (torch.triu(torch.ones((size, size), device=device)) == 1).transpose(0, 1)
    mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
    return mask

In [14]:
def create_mask(src, tgt, pad_idx, device):

    src_seq_len = src.shape[0]
    tgt_seq_len = tgt.shape[0]

    tgt_mask = generate_square_subsequent_mask(tgt_seq_len, device)
    src_mask = torch.zeros((src_seq_len, src_seq_len),device=device).type(torch.bool)

    src_padding_mask = (src == pad_idx).transpose(0, 1)
    tgt_padding_mask = (tgt == pad_idx).transpose(0, 1)
    return src_mask, tgt_mask, src_padding_mask, tgt_padding_mask

In [15]:
def train(model, train_dl, loss_fn, optim, special_symbols):
    losses = 0

    model.train()
    for src, tgt in tqdm(train_dl, ascii=True):
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)
        tgt_input = tgt[:-1, :]
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input, special_symbols["<pad>"], DEVICE)
        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)
        optim.zero_grad()
        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        loss.backward()
        optim.step()
        losses += loss.item()
    return losses / len(list(train_dl))

In [16]:
def validate(model, valid_dl, loss_fn, special_symbols):
    losses = 0
    model.eval()

    for src, tgt in tqdm(valid_dl):
        src = src.to(DEVICE)
        tgt = tgt.to(DEVICE)
        tgt_input = tgt[:-1, :]
        src_mask, tgt_mask, src_padding_mask, tgt_padding_mask = create_mask(src, tgt_input, special_symbols["<pad>"], DEVICE)
        logits = model(src, tgt_input, src_mask, tgt_mask,src_padding_mask, tgt_padding_mask, src_padding_mask)
        tgt_out = tgt[1:, :]
        loss = loss_fn(logits.reshape(-1, logits.shape[-1]), tgt_out.reshape(-1))
        losses += loss.item()

    return losses / len(list(valid_dl))

In [17]:
train_dl, valid_dl, src_vocab, tgt_vocab, src_lang_transform, tgt_lang_transform, special_symbols = get_data(train_src, train_tgt, val_src, val_tgt, test_data)

src_vocab_size = len(src_vocab)
print(src_vocab_size)
tgt_vocab_size = len(tgt_vocab)
print(tgt_vocab_size)
    
attn_heads=8
enc_layers=3
dec_layers=3
                        
embed_size=512           
dim_feedforward=512
dropout = 0.15
    
epochs = 50
    
model = Translator(
        num_encoder_layers=enc_layers,
        num_decoder_layers=dec_layers,
        embed_size=embed_size,
        num_heads=attn_heads,
        src_vocab_size=src_vocab_size,
        tgt_vocab_size=tgt_vocab_size,
        dim_feedforward=dim_feedforward,
        dropout=dropout
    ).to(DEVICE)
    
loss_fn = torch.nn.CrossEntropyLoss(ignore_index=special_symbols["<pad>"], label_smoothing=0.1)
    
optim = torch.optim.Adam(model.parameters(), lr=1e-4, betas=(0.9, 0.98), eps=1e-9)

25379
19057


/usr/local/lib/python3.10/dist-packages/torch/nn/modules/transformer.py:306: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


In [18]:
#model.load_state_dict(torch.load('/kaggle/input/transformer_long/pytorch/default/1/transformer_long_3.pth'))

<All keys matched successfully>

In [19]:
for idx, epoch in enumerate(range(1, epochs+1)):
    start_time = time()
    train_loss = train(model, train_dl, loss_fn, optim, special_symbols)
    torch.save(model.state_dict(), 'transformer_long_3.pth')
    epoch_time = time() - start_time
    val_loss   = validate(model, valid_dl, loss_fn, special_symbols)
    wandb.log({"epoch": epoch, "train_loss": train_loss, "val_loss": val_loss})
    print(f"Epoch: {epoch}\n\tTrain loss: {train_loss:.3f}\n\tVal loss: {val_loss:.3f}\n\tEpoch time = {epoch_time:.1f} seconds\n\tETA = {epoch_time*(epochs-idx-1):.1f} seconds")

  0%|          | 0/1531 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/functional.py:5137: UserWarning: Support for mismatched key_padding_mask and attn_mask is deprecated. Use same type for both instead.
  warnings.warn(
100%|██████████| 8/8 [00:01<00:00,  6.51it/s]


Epoch: 1
	Train loss: 6.347
	Val loss: 5.838
	Epoch time = 689.3 seconds
	ETA = 33777.6 seconds


100%|██████████| 8/8 [00:01<00:00,  6.51it/s]


Epoch: 2
	Train loss: 5.745
	Val loss: 5.526
	Epoch time = 697.2 seconds
	ETA = 33466.3 seconds


100%|██████████| 8/8 [00:01<00:00,  6.43it/s]


...

Epoch: 48
	Train loss: 3.164
	Val loss: 3.125
	Epoch time = 697.3 seconds
	ETA = 1394.6 seconds


100%|██████████| 8/8 [00:01<00:00,  6.45it/s]


Epoch: 49
	Train loss: 3.156
	Val loss: 3.122
	Epoch time = 697.5 seconds
	ETA = 697.5 seconds


100%|██████████| 8/8 [00:01<00:00,  6.54it/s]

Epoch: 50
	Train loss: 3.148
	Val loss: 3.115
	Epoch time = 698.0 seconds
	ETA = 0.0 seconds


In [19]:
def greedy_decode(model, src, src_mask, max_len, start_symbol, end_symbol):
    src = src.to(DEVICE)
    src_mask = src_mask.to(DEVICE)

    memory = model.encode(src, src_mask)

    ys = torch.ones(1, 1).fill_(start_symbol).type(torch.long).to(DEVICE)

    for _ in range(max_len-1):

        memory = memory.to(DEVICE)
        tgt_mask = (generate_square_subsequent_mask(ys.size(0), DEVICE).type(torch.bool)).to(DEVICE)
        out = model.decode(ys, memory, tgt_mask)

        out = out.transpose(0, 1)

        prob = model.ff(out[:, -1])
        _, next_word = torch.max(prob, dim=1)
        next_word = next_word.item()

        ys = torch.cat([ys, torch.ones(1, 1).type_as(src.data).fill_(next_word)], dim=0)
        if next_word == end_symbol:
            break

    return ys

In [44]:
def inference(model, train_dl, valid_dl, src_vocab, tgt_vocab, src_lang_transform, tgt_lang_transform, special_symbols):
    model.eval()

    translated_sentences = []

    for sentence in tqdm(test_data):
       
        encoded_src = src_lang_transform(sentence)
        decoded_src = [src_vocab.lookup_tokens([token])[0] for token in encoded_src]
        
        unk_words = [word for word in sentence if word not in decoded_src]

        src = encoded_src.view(-1, 1)
        num_tokens = src.shape[0]
        src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)

        tgt_tokens = greedy_decode(
            model, src, src_mask, max_len=num_tokens+5, start_symbol=special_symbols["<bos>"], end_symbol=special_symbols["<eos>"]
        ).flatten()

        output_as_list = list(tgt_tokens.cpu().numpy())
        output_list_words = tgt_vocab.lookup_tokens(output_as_list)

        unk_index = 0
        final_translation = []
        for word in output_list_words:
            if word == "<bos>" or word == "<eos>":
                continue 
            if word == "<unk>" and unk_index < len(unk_words):
                final_translation.append(unk_words[unk_index])  
                unk_index += 1
            else:
                final_translation.append(word)

        translated_sentences.append(" ".join(final_translation).strip())

    with open("translated_test.txt", "w", encoding="utf-8") as f:
        for line in translated_sentences:
            f.write(line + "\n")

    print("Translation complete. Saved to translated_test.txt")


In [45]:
inference(model, train_dl, valid_dl, src_vocab, tgt_vocab, src_lang_transform, tgt_lang_transform, special_symbols)

100%|██████████| 2998/2998 [04:03<00:00, 12.30it/s]

Translation complete. Saved to translated_test.txt


In [23]:
# def inference():

#     train_dl, valid_dl, src_vocab, tgt_vocab, src_lang_transform, tgt_lang_transform, special_symbols = get_data(train_src, train_tgt, val_src, val_tgt, test_data)

#     src_vocab_size = len(src_vocab)
#     tgt_vocab_size = len(tgt_vocab)
    
#     attn_heads=8
#     enc_layers=5
#     dec_layers=5
                        
#     embed_size=512           
#     dim_feedforward=512
#     dropout = 0.3
 
#     model = Translator(
#         num_encoder_layers=enc_layers,
#         num_decoder_layers=dec_layers,
#         embed_size=embed_size,
#         num_heads=attn_heads,
#         src_vocab_size=src_vocab_size,
#         tgt_vocab_size=tgt_vocab_size,
#         dim_feedforward=dim_feedforward,
#         dropout=dropout
#     ).to(DEVICE)

#     # Load in weights
#     #model.load_state_dict(torch.load(opts.model_path))

#     # Set to inference
#     model.eval()

#     # Accept input and keep translating until they quit
#     while True:
#         print("> ", end="")

#         sentence = input()

#         # Convert to tokens
#         src = src_transform(sentence).view(-1, 1)
#         num_tokens = src.shape[0]

#         src_mask = (torch.zeros(num_tokens, num_tokens)).type(torch.bool)

#         # Decode
#         tgt_tokens = greedy_decode(
#             model, src, src_mask, max_len=num_tokens+5, start_symbol=special_symbols["<bos>"], end_symbol=special_symbols["<eos>"]
#         ).flatten()

#         # Convert to list of tokens
#         output_as_list = list(tgt_tokens.cpu().numpy())

#         # Convert tokens to words
#         output_list_words = tgt_vocab.lookup_tokens(output_as_list)

#         # Remove special tokens and convert to string
#         translation = " ".join(output_list_words).replace("<bos>", "").replace("<eos>", "")

#         print(translation)